In [1]:
!pip install rapidfuzz


In [4]:
# =========================================================
# 📘 01_data_preprocessing.ipynb (Final Optimized Version)
# Author: Apoorva Sharma
# Project: AI-Powered Precision Nutrition
# Purpose: Combine and enrich recipe + ingredient data with USDA, environment & cost
# =========================================================

import pandas as pd
import numpy as np
import json
import re
from pathlib import Path

# ---------------------------------------------------------
# 1️⃣ Setup paths
# ---------------------------------------------------------
BASE_DIR = Path("D:/Complete_Data/ml_part_nutrition_project")
DATA_DIR = BASE_DIR / "Datasets"
RECIPE_DIR = DATA_DIR / "recipe_dataset"
ENV_DIR = DATA_DIR / "environment_dataset"
COST_DIR = DATA_DIR / "cost_dataset"
USDA_DIR = DATA_DIR / "FoodData_Central_sr_legacy_food_csv_2018-04"
PROCESSED_DIR = BASE_DIR / "processed_data"

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Input paths
RECIPES_JSON = RECIPE_DIR / "full_format_recipes.json"
INGREDIENTS_PATH = RECIPE_DIR / "ingredients.csv"
USDA_FOOD_PATH = USDA_DIR / "sr_legacy_food.csv"
USDA_NUTR_PATH = USDA_DIR / "food_nutrient.csv"
ENV_PATH = ENV_DIR / "Food_production.csv"
COST_PATH = COST_DIR / "wfp_food_prices_ind.csv"

# ---------------------------------------------------------
# 2️⃣ Load datasets
# ---------------------------------------------------------
print("📥 Loading datasets...")

ingredients = pd.read_csv(INGREDIENTS_PATH)
usda_food = pd.read_csv(USDA_FOOD_PATH)
usda_nutr = pd.read_csv(USDA_NUTR_PATH)
env_data = pd.read_csv(ENV_PATH)
cost_data = pd.read_csv(COST_PATH)

with open(RECIPES_JSON, "r", encoding="utf-8") as f:
    recipes_json = json.load(f)

print(f"✅ Loaded ingredients: {ingredients.shape}")
print(f"✅ Loaded USDA food: {usda_food.shape}")
print(f"✅ Loaded USDA nutrient: {usda_nutr.shape}")
print(f"✅ Loaded environment: {env_data.shape}")
print(f"✅ Loaded cost data: {cost_data.shape}")
print(f"✅ Loaded recipes JSON: {len(recipes_json)} recipes")

# ---------------------------------------------------------
# 3️⃣ Text Cleaning Utilities
# ---------------------------------------------------------
def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def rapid_match(word, choices, threshold=0.8):
    """
    A faster approximate matching using substring containment
    instead of fuzzywuzzy (much faster for large datasets).
    """
    if not isinstance(word, str) or not word.strip():
        return None
    for c in choices:
        if len(word) > 3 and (word in c or c in word):
            return c
    return None

# Clean ingredient names
ingredients["ingredient_clean"] = ingredients["ingredient_raw"].apply(clean_text)

# ---------------------------------------------------------
# 4️⃣ Extract recipe titles & ingredients from JSON (Auto-Detect Format)
# ---------------------------------------------------------
print("🔍 Parsing recipe JSON...")

def extract_recipes_auto(recipes_json):
    # Case 1: JSON list of dictionaries with 'title' and 'ingredients'
    if isinstance(recipes_json, list) and len(recipes_json) > 0:
        first_item = recipes_json[0]
        
        # Case 1a: Deep dict structure (with ingredient text objects)
        if isinstance(first_item, dict) and "ingredients" in first_item and isinstance(first_item["ingredients"], list):
            return pd.DataFrame([
                {
                    "recipe_id": r.get("id", None),
                    "recipe_title": r.get("title", f"Recipe_{i}"),
                    "ingredient_text": ", ".join([
                        clean_text(i.get("text", "")) if isinstance(i, dict) else clean_text(str(i))
                        for i in r.get("ingredients", [])
                    ]),
                }
                for i, r in enumerate(recipes_json)
            ])

        # Case 1b: Simplified dict structure with string lists
        elif isinstance(first_item, dict) and "ingredients" in first_item and isinstance(first_item["ingredients"], list):
            return pd.DataFrame([
                {
                    "recipe_id": r.get("id", f"R_{i}"),
                    "recipe_title": r.get("title", f"Recipe_{i}"),
                    "ingredient_text": ", ".join([clean_text(str(x)) for x in r.get("ingredients", [])]),
                }
                for i, r in enumerate(recipes_json)
            ])

        # Case 1c: Dictionary without proper keys (flatten)
        elif isinstance(first_item, dict):
            return pd.DataFrame([
                {
                    "recipe_id": r.get("id", f"R_{i}"),
                    "recipe_title": r.get("title", f"Recipe_{i}"),
                    "ingredient_text": clean_text(str(r)),
                }
                for i, r in enumerate(recipes_json)
            ])

        # Case 2: JSON list of strings
        elif isinstance(first_item, str):
            return pd.DataFrame({
                "recipe_id": range(len(recipes_json)),
                "recipe_title": [f"Recipe_{i}" for i in range(len(recipes_json))],
                "ingredient_text": [clean_text(r) for r in recipes_json],
            })

    # Fallback: return empty DataFrame
    return pd.DataFrame(columns=["recipe_id", "recipe_title", "ingredient_text"])


recipes_df = extract_recipes_auto(recipes_json)
print(f"✅ Extracted recipes: {recipes_df.shape}")

# ---------------------------------------------------------
# 5️⃣ Map USDA nutrient info
# ---------------------------------------------------------
CORE_NUTRIENTS = {
    1008: "energy_kcal",
    1003: "protein_g",
    1004: "fat_g",
    1005: "carbs_g",
}

usda_nutr = usda_nutr[usda_nutr["nutrient_id"].isin(CORE_NUTRIENTS.keys())].copy()
usda_nutr["nutrient_name"] = usda_nutr["nutrient_id"].map(CORE_NUTRIENTS)

usda_pivot = (
    usda_nutr.pivot_table(
        index="fdc_id",
        columns="nutrient_name",
        values="amount",
        aggfunc="mean"
    ).reset_index()
)

usda_combined = pd.merge(usda_food, usda_pivot, on="fdc_id", how="left")

# Identify description column
desc_col = [c for c in usda_combined.columns if "desc" in c.lower() or "food" in c.lower()]
if desc_col:
    usda_combined["description_clean"] = usda_combined[desc_col[0]].apply(clean_text)
else:
    usda_combined["description_clean"] = usda_combined["NDB_number"].astype(str).apply(clean_text)

print("✅ USDA combined nutrient table:", usda_combined.shape)

# ---------------------------------------------------------
# 6️⃣ Rapid match ingredients → USDA (nutrition)
# ---------------------------------------------------------
usda_choices = usda_combined["description_clean"].dropna().unique().tolist()
ingredients["usda_match"] = ingredients["ingredient_clean"].apply(
    lambda x: rapid_match(x, usda_choices, threshold=0.8)
)

ingredients = ingredients.merge(
    usda_combined,
    left_on="usda_match",
    right_on="description_clean",
    how="left"
)
print("✅ Rapid matched USDA nutrient data")

# ---------------------------------------------------------
# 7️⃣ Rapid match environment data
# ---------------------------------------------------------
env_data["Food product_clean"] = env_data["Food product"].apply(clean_text)
env_choices = env_data["Food product_clean"].dropna().unique().tolist()

ingredients["env_match"] = ingredients["ingredient_clean"].apply(
    lambda x: rapid_match(x, env_choices, threshold=0.8)
)

ingredients = ingredients.merge(
    env_data[["Food product_clean", "Total_emissions", "Land use change"]],
    left_on="env_match",
    right_on="Food product_clean",
    how="left"
)
print("✅ Rapid matched environmental data")

# ---------------------------------------------------------
# 8️⃣ Clean & match cost data (fixed numeric handling)
# ---------------------------------------------------------
print("🔹 Cleaning cost data before averaging...")

cost_data["commodity_clean"] = cost_data["commodity"].apply(clean_text)
cost_data["usdprice"] = (
    cost_data["usdprice"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.extract(r"(\d+\.?\d*)")[0]
)
cost_data["usdprice"] = pd.to_numeric(cost_data["usdprice"], errors="coerce")
cost_data = cost_data.dropna(subset=["usdprice"])

avg_cost = cost_data.groupby("commodity_clean", as_index=False)["usdprice"].mean()
cost_choices = avg_cost["commodity_clean"].dropna().unique().tolist()

ingredients["cost_match"] = ingredients["ingredient_clean"].apply(
    lambda x: rapid_match(x, cost_choices, threshold=0.8)
)

ingredients = ingredients.merge(
    avg_cost, left_on="cost_match", right_on="commodity_clean", how="left"
)
ingredients.rename(columns={"usdprice": "price"}, inplace=True)
print("✅ Rapid matched cost data (numeric-safe)")

# ---------------------------------------------------------
# 9️⃣ Save combined ingredient-level dataset
# ---------------------------------------------------------
ingredient_save_path = PROCESSED_DIR / "ingredients_with_nutrients_cost_env.csv"
ingredients.to_csv(ingredient_save_path, index=False)
print(f"💾 Saved enriched ingredient dataset → {ingredient_save_path}")

# ---------------------------------------------------------
# 🔟 Aggregate recipe-level nutrients
# ---------------------------------------------------------
nutr_cols = ["energy_kcal", "protein_g", "fat_g", "carbs_g", "price", "Total_emissions"]
nutr_cols = [c for c in nutr_cols if c in ingredients.columns]

recipe_summary = (
    ingredients.groupby("recipe_title")[nutr_cols]
    .mean(numeric_only=True)
    .reset_index()
    .rename(columns={c: f"{c}_mean" for c in nutr_cols})
)

recipes_enriched = pd.merge(recipes_df, recipe_summary, on="recipe_title", how="left")
recipes_enriched.to_csv(PROCESSED_DIR / "recipes_enriched.csv", index=False)

print(f"✅ Final enriched recipes saved → {PROCESSED_DIR / 'recipes_enriched.csv'}")
print(f"📊 Shape: {recipes_enriched.shape}")


📥 Loading datasets...


C:\Users\Apoorva\AppData\Local\Temp\ipykernel_13188\1396067817.py:44: DtypeWarning: Columns (4,5,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  cost_data = pd.read_csv(COST_PATH)


✅ Loaded ingredients: (199106, 3)
✅ Loaded USDA food: (7793, 2)
✅ Loaded USDA nutrient: (644125, 11)
✅ Loaded environment: (43, 23)
✅ Loaded cost data: (172018, 14)
✅ Loaded recipes JSON: 20130 recipes
🔍 Parsing recipe JSON...
✅ Extracted recipes: (20130, 3)
✅ USDA combined nutrient table: (7793, 7)
✅ Rapid matched USDA nutrient data
✅ Rapid matched environmental data
🔹 Cleaning cost data before averaging...
✅ Rapid matched cost data (numeric-safe)
💾 Saved enriched ingredient dataset → D:\Complete_Data\ml_part_nutrition_project\processed_data\ingredients_with_nutrients_cost_env.csv
✅ Final enriched recipes saved → D:\Complete_Data\ml_part_nutrition_project\processed_data\recipes_enriched.csv
📊 Shape: (20130, 9)


In [6]:
# 02_environment_mapping_fast.ipynb
# Purpose: map environmental metrics (emissions, land use, water, ...) to ingredients,
# and aggregate to recipe-level. Uses difflib for fast approximate matching.
# Author: adapted for Apoorva Sharma project

import pandas as pd
import numpy as np
import difflib
from pathlib import Path
import re
import math
import time

# ----------------------------
# 0. Paths & config (edit if needed)
# ----------------------------
BASE_DIR = Path("D:/Complete_Data/ml_part_nutrition_project")
DATASETS_DIR = BASE_DIR / "Datasets"
PROCESSED_DIR = BASE_DIR / "processed_data"
ENV_DIR = DATASETS_DIR / "environment_dataset"

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Ingredient-level file produced earlier by preprocessing step
INGREDIENTS_PATH_CANDIDATES = [
    PROCESSED_DIR / "ingredients_with_nutrients_cost_env.csv",
    PROCESSED_DIR / "ingredients_with_nutrients.csv",
    BASE_DIR / "processed_data" / "ingredients_with_nutrients_cost_env.csv"
]

# Environment data file (your Food_Production / environment dataset)
ENV_CANDIDATES = [
    ENV_DIR / "Food_production.csv",
    ENV_DIR / "Food_Production.csv",
    DATASETS_DIR / "environment_dataset" / "Food_Production.csv",
    DATASETS_DIR / "environment_dataset" / "Food_production.csv"
]

# Output filenames
OUT_ING = PROCESSED_DIR / "ingredients_with_env_mapped.csv"
OUT_REC = PROCESSED_DIR / "recipes_with_env_metrics.csv"

# similarity threshold for difflib.get_close_matches (0..1 scaled later)
MATCH_CUTOFF = 0.70   # conservative; adjust 0.6..0.8 as needed
MAX_CANDIDATES = 5    # how many close matches to examine

# Batch size for applying mapping to avoid memory spikes
BATCH_SIZE = 20000

# ----------------------------
# 1. Helper functions
# ----------------------------
def normalize_text(s):
    """Lowercase, strip punctuation and extra spaces."""
    if pd.isna(s):
        return ""
    s = str(s).lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def find_best_match(name, choices_list, cutoff=MATCH_CUTOFF):
    """
    Use difflib.get_close_matches to find best approximate match.
    Returns matched choice string or None.
    """
    if not name or not isinstance(name, str):
        return None
    # difflib works on raw strings; normalize externally
    candidates = difflib.get_close_matches(name, choices_list, n=MAX_CANDIDATES, cutoff=cutoff)
    return candidates[0] if candidates else None

def safe_float(x):
    try:
        return float(x)
    except Exception:
        return np.nan

# ----------------------------
# 2. Load files (robust selection)
# ----------------------------
# Ingredients file
INGREDIENTS_PATH = None
for p in INGREDIENTS_PATH_CANDIDATES:
    if p.exists():
        INGREDIENTS_PATH = p
        break
if INGREDIENTS_PATH is None:
    raise FileNotFoundError(f"Could not find ingredients file. Looked at: {INGREDIENTS_PATH_CANDIDATES}")

print("Loading ingredients from:", INGREDIENTS_PATH)
ingredients = pd.read_csv(INGREDIENTS_PATH, low_memory=False)
print("Loaded ingredients:", ingredients.shape)
print("Columns:", list(ingredients.columns)[:40])

# Environment file
ENV_PATH = None
for p in ENV_CANDIDATES:
    if p.exists():
        ENV_PATH = p
        break
if ENV_PATH is None:
    raise FileNotFoundError(f"Could not find environment file. Looked at: {ENV_CANDIDATES}")

print("Loading environment data from:", ENV_PATH)
env_df = pd.read_csv(ENV_PATH, low_memory=False)
print("Loaded environment:", env_df.shape)
print("Environment columns:", list(env_df.columns)[:60])

# ----------------------------
# 3. Identify useful env metric columns
# ----------------------------
env_col_candidates = [c for c in env_df.columns if any(k in c.lower() for k in (
    "emission", "total_emissions", "total_emission", "greenhouse", "ghg", "co2",
    "land", "land use", "landuse", "land_use", "land per", "eutroph", "water", "freshwater"))]

# Also show a short list to the user for verification
print("Detected environment-like columns (candidates):")
for c in env_col_candidates[:40]:
    print("  -", c)

# Choose the most likely ones
preferred_emission_cols = [c for c in env_col_candidates if "emiss" in c.lower() or "total_emissions" in c.lower() or "greenhouse" in c.lower()]
preferred_land_cols = [c for c in env_col_candidates if "land" in c.lower()]

emission_col = preferred_emission_cols[0] if preferred_emission_cols else (env_col_candidates[0] if env_col_candidates else None)
land_col = preferred_land_cols[0] if preferred_land_cols else None

print("Selected emission column:", emission_col)
print("Selected land column:", land_col)

# If none found, list some manual column names for user to update
if emission_col is None:
    raise ValueError("No emission-like column detected in environment dataset. Please inspect `env_df.columns` and update the script.")

# ----------------------------
# 4. Prepare matching keys
# ----------------------------
# find plausable product name column in env_df
possible_name_cols = [c for c in env_df.columns if any(k in c.lower() for k in ("food product","food","product","commodity","item","name"))]
env_name_col = possible_name_cols[0] if possible_name_cols else env_df.columns[0]
print("Using environment name column:", env_name_col)

# create cleaned name column in environment dataset
env_df["env_name_clean"] = env_df[env_name_col].astype(str).apply(normalize_text)

# create choices list (unique)
env_choices = env_df["env_name_clean"].dropna().unique().tolist()
print(f"Unique env choices: {len(env_choices)}")

# prepare ingredient names to match
# find a good ingredient text column in ingredients file
possible_ing_cols = [c for c in ingredients.columns if any(k in c.lower() for k in ("ingredient_clean","ingredient_text","ingredient_raw","ingredient"))]
if possible_ing_cols:
    ing_col = possible_ing_cols[0]
else:
    # fallback - pick the first column that looks like string
    ing_col = ingredients.columns[0]

print("Using ingredient column:", ing_col)

# ensure we have a cleaned ingredient column
if "ingredient_clean" not in ingredients.columns:
    ingredients["ingredient_clean"] = ingredients[ing_col].astype(str).apply(normalize_text)
else:
    # normalize existing column
    ingredients["ingredient_clean"] = ingredients["ingredient_clean"].astype(str).apply(normalize_text)

# ----------------------------
# 5. Rapid mapping (batched) from ingredient -> env record
# ----------------------------
start_time = time.time()
print("Mapping ingredients to environment rows (batched)...")

# Pre-memoize mapping for unique ingredient strings to avoid repeated matching
unique_ings = ingredients["ingredient_clean"].dropna().unique().tolist()
print("Unique ingredients count:", len(unique_ings))

# build a dict of mapping for unique names using difflib
mapping = {}
for i, name in enumerate(unique_ings):
    # quick heuristic: if name appears in any env choice as substring, prefer that
    if not name:
        mapping[name] = None
        continue

    # substring direct match
    substr_matches = [c for c in env_choices if name in c or c in name]
    if substr_matches:
        mapping[name] = substr_matches[0]
        continue

    # difflib approximate match
    best = find_best_match(name, env_choices, cutoff=MATCH_CUTOFF)
    mapping[name] = best

    # progress print occasionally
    if (i+1) % 5000 == 0:
        print(f"  matched {i+1}/{len(unique_ings)} unique ingredients...")

print("Mapping build done in {:.1f}s".format(time.time() - start_time))

# apply mapping to ingredients in batches (to avoid huge intermediate arrays)
mapped_env_names = []
for start in range(0, len(ingredients), BATCH_SIZE):
    batch = ingredients["ingredient_clean"].iloc[start:start+BATCH_SIZE].tolist()
    mapped_batch = [mapping.get(x, None) for x in batch]
    mapped_env_names.extend(mapped_batch)
ingredients["env_match"] = mapped_env_names

# ----------------------------
# 6. Join selected environment metrics
# ----------------------------
# choose subset of env_df to merge
env_subset = env_df[["env_name_clean", emission_col] + ([land_col] if land_col else [])].drop_duplicates(subset=["env_name_clean"])
# ensure numeric
env_subset[emission_col] = pd.to_numeric(env_subset[emission_col], errors="coerce")
if land_col:
    env_subset[land_col] = pd.to_numeric(env_subset[land_col], errors="coerce")

ingredients = ingredients.merge(env_subset, left_on="env_match", right_on="env_name_clean", how="left", suffixes=("", "_env"))

# rename to unified column names
ingredients = ingredients.rename(columns={emission_col: "Total_emissions", land_col: "Land_use_change"} if land_col else {emission_col: "Total_emissions"})

# ----------------------------
# 7. Fallbacks / cleaning
# ----------------------------
# If some ingredients missing emissions, fill with median of mapped values
if "Total_emissions" in ingredients.columns:
    median_em = ingredients["Total_emissions"].median(skipna=True)
    ingredients["Total_emissions"] = ingredients["Total_emissions"].fillna(median_em)
else:
    ingredients["Total_emissions"] = np.nan

if "Land_use_change" in ingredients.columns:
    median_land = ingredients["Land_use_change"].median(skipna=True)
    ingredients["Land_use_change"] = ingredients["Land_use_change"].fillna(median_land)

# Save ingredient-level mapping
ingredients.to_csv(OUT_ING, index=False)
print("Saved ingredient-level env mapping to:", OUT_ING)
print("Ingredients shape now:", ingredients.shape)

# ----------------------------
# 8. Aggregate per recipe
# ----------------------------
# Determine the recipe title/key column in ingredients. Try common names.
possible_recipe_cols = [c for c in ingredients.columns if any(k in c.lower() for k in ("recipe_title","recipe_id","title","recipe"))]
recipe_key_col = possible_recipe_cols[0] if possible_recipe_cols else None

if recipe_key_col is None:
    # Try to join with recipes file in processed_data if exists
    print("No recipe_key in ingredients — attempting to find recipes file to map by index/title...")
    recipe_candidates = [
        PROCESSED_DIR / "recipes_enriched.csv",
        PROCESSED_DIR / "recipes_master.csv",
        PROCESSED_DIR / "recipes_enriched_v2.csv"
    ]
    recipes_df = None
    for p in recipe_candidates:
        if p.exists():
            recipes_df = pd.read_csv(p, low_memory=False)
            print("Loaded recipes file for aggregation:", p)
            break
    if recipes_df is None:
        # can't aggregate by recipe; simply compute ingredient-level metrics and finish
        print("Could not find recipes file; skipping recipe aggregation. You can aggregate externally.")
        recipes_agg = pd.DataFrame()  # empty
    else:
        # try to merge by a likely field (title vs recipe_title)
        if "title" in recipes_df.columns:
            # build mapping from recipe title -> ingredients (if ingredients has info to join)
            # if ingredients had a recipe_title column earlier it'll be used; else we can't map reliably
            if "recipe_title" in ingredients.columns:
                recipes_agg = ingredients.groupby("recipe_title")[["Total_emissions", "Land_use_change"]].mean().reset_index()
            else:
                # attempt to derive mapping from ingredients file: see if ingredients has 'recipe_id' or other id columns
                possible_id_cols = [c for c in ingredients.columns if "id" in c.lower()]
                if possible_id_cols:
                    idcol = possible_id_cols[0]
                    recipes_agg = ingredients.groupby(idcol)[["Total_emissions", "Land_use_change"]].mean().reset_index().rename(columns={idcol: "recipe_id"})
                else:
                    print("No recipe linking column found inside ingredients. Skipping recipe aggregation.")
                    recipes_agg = pd.DataFrame()
else:
    # standard path: aggregate grouped by recipe key present in ingredients
    recipes_agg = ingredients.groupby(recipe_key_col)[["Total_emissions", "Land_use_change"]].mean().reset_index()
    recipes_agg = recipes_agg.rename(columns={recipe_key_col: "recipe_title"})

# Save recipe-level aggregation if not empty
if not recipes_agg.empty:
    recipes_agg.to_csv(OUT_REC, index=False)
    print("Saved recipe-level env metrics to:", OUT_REC)
    print("Recipe-agg shape:", recipes_agg.shape)
else:
    print("Recipe aggregation skipped or no mapping available. You still have ingredient-level file:", OUT_ING)

# ----------------------------
# 9. Summary
# ----------------------------
print("\nDone. Summary:")
print("Ingredients file:", OUT_ING)
if not recipes_agg.empty:
    print("Recipe metrics file:", OUT_REC)
print("Time elapsed: {:.1f}s".format(time.time() - start_time))


Loading ingredients from: D:\Complete_Data\ml_part_nutrition_project\processed_data\ingredients_with_nutrients_cost_env.csv
Loaded ingredients: (199106, 19)
Columns: ['recipe_id', 'recipe_title', 'ingredient_raw', 'ingredient_clean', 'usda_match', 'fdc_id', 'NDB_number', 'carbs_g', 'energy_kcal', 'fat_g', 'protein_g', 'description_clean', 'env_match', 'Food product_clean', 'Total_emissions', 'Land use change', 'cost_match', 'commodity_clean', 'price']
Loading environment data from: D:\Complete_Data\ml_part_nutrition_project\Datasets\environment_dataset\Food_production.csv
Loaded environment: (43, 23)
Environment columns: ['Food product', 'Land use change', 'Animal Feed', 'Farm', 'Processing', 'Transport', 'Packging', 'Retail', 'Total_emissions', 'Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal)', 'Eutrophying emissions per kilogram (gPO₄eq per kilogram)', 'Eutrophying emissions per 100g protein (gPO₄eq per 100 grams protein)', 'Freshwater withdrawals per 1000kcal (liters per 10